In [3]:
from zeep import Client
from zeep.wsse.username import UsernameToken
from base64 import b64decode
from zipfile import ZipFile
import pandas as pd
import numpy as np
import csv
import io
import json
import os, time, datetime
from datetime import datetime
from time import perf_counter 

In [2]:
!pip install zeep

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.2/101.2 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 12.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.7/121.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 7.1 MB/s eta 0:00:00
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)
  DEPRECATION: lxml is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.

In [4]:
dict_creden_TERYT = {'username' : 'TestPubliczny',
                    'password' : '1234abcd',
                    'wsdl' : 'https://uslugaterytws1test.stat.gov.pl/wsdl/terytws1.wsdl'}

In [6]:
token = UsernameToken(
    username = dict_creden_TERYT['username'],
    password = dict_creden_TERYT['password']
    )

In [7]:
token

In [9]:
client = Client(
    wsdl = dict_creden_TERYT['wsdl'],
    wsse = token
    )

In [10]:
# setting parameters for api
STATE_DATE = datetime.now()
catalog = client.service.PobierzKatalogTERC(STATE_DATE)
filename = catalog['nazwa_pliku']
content = catalog['plik_zawartosc']
decoded = b64decode(content)

In [12]:
with open(filename, "wb") as file:
    file.write(decoded)
    file.close()

In [13]:
zf = ZipFile(filename, 'r')
zf

<zipfile.ZipFile filename='TERC_Urzedowy_2023-01-01' mode='r'>

In [14]:
with zf.open(zf.namelist()[1]) as csv_file:
    data = pd.read_csv(csv_file, 
                        sep=';', 
                        quoting=csv.QUOTE_NONE, 
                        encoding='utf-8',
                        dtype={
                            'WOJ' : str,
                            'POW' : str, 
                            'GMI' : str, 
                            'RODZ' : str, 
                            'NAZWA' : str, 
                            'STAN_NA'  : str       
                         })

In [17]:
data

,WOJ,POW,GMI,RODZ,NAZWA,NAZWA_DOD,STAN_NA
0,02,NaN,NaN,NaN,DOLNOŚLĄSKIE,województwo,2023-01-01
1,02,01,NaN,NaN,bolesławiecki,powiat,2023-01-01
2,02,01,01,1,Bolesławiec,gmina miejska,2023-01-01
3,02,01,02,2,Bolesławiec,gmina wiejska,2023-01-01
4,02,01,03,2,Gromadka,gmina wiejska,2023-01-01
...,...,...,...,...,...,...,...
4259,32,61,01,1,Koszalin,gmina miejska,2023-01-01
4260,32,62,NaN,NaN,Szczecin,miasto na prawach powiatu,2023-01-01
4261,32,62,01,1,Szczecin,gmina miejska,2023-01-01
4262,32,63,NaN,NaN,Świnoujście,miasto na prawach powiatu,2023-01-01


In [18]:
type(data)

pandas.core.frame.DataFrame

In [19]:
data_terc = data.copy()

In [20]:
data_terc = data_terc.replace(np.nan, "")

In [21]:
data_terc

,WOJ,POW,GMI,RODZ,NAZWA,NAZWA_DOD,STAN_NA
0,02,,,,DOLNOŚLĄSKIE,województwo,2023-01-01
1,02,01,,,bolesławiecki,powiat,2023-01-01
2,02,01,01,1,Bolesławiec,gmina miejska,2023-01-01
3,02,01,02,2,Bolesławiec,gmina wiejska,2023-01-01
4,02,01,03,2,Gromadka,gmina wiejska,2023-01-01
...,...,...,...,...,...,...,...
4259,32,61,01,1,Koszalin,gmina miejska,2023-01-01
4260,32,62,,,Szczecin,miasto na prawach powiatu,2023-01-01
4261,32,62,01,1,Szczecin,gmina miejska,2023-01-01
4262,32,63,,,Świnoujście,miasto na prawach powiatu,2023-01-01


In [22]:
voivodeship = data_terc[data_terc.NAZWA_DOD == 'województwo'].copy()

In [23]:
voivodeship

,WOJ,POW,GMI,RODZ,NAZWA,NAZWA_DOD,STAN_NA
0,02,,,,DOLNOŚLĄSKIE,województwo,2023-01-01
321,04,,,,KUJAWSKO-POMORSKIE,województwo,2023-01-01
561,06,,,,LUBELSKIE,województwo,2023-01-01
861,08,,,,LUBUSKIE,województwo,2023-01-01
1026,10,,,,ŁÓDZKIE,województwo,2023-01-01
1301,12,,,,MAŁOPOLSKIE,województwo,2023-01-01
1610,14,,,,MAZOWIECKIE,województwo,2023-01-01
2113,16,,,,OPOLSKIE,województwo,2023-01-01
2263,18,,,,PODKARPACKIE,województwo,2023-01-01
2521,20,,,,PODLASKIE,województwo,2023-01-01


In [24]:
voivodeship_df = data_terc[data_terc.NAZWA_DOD == "województwo"].copy().drop(['POW',
                                                                             'GMI', 
                                                                             'RODZ',
                                                                             'NAZWA_DOD',
                                                                             'STAN_NA'], axis=1).reset_index(drop=True)
voivodeship_df

,WOJ,NAZWA
0,02,DOLNOŚLĄSKIE
1,04,KUJAWSKO-POMORSKIE
2,06,LUBELSKIE
3,08,LUBUSKIE
4,10,ŁÓDZKIE
5,12,MAŁOPOLSKIE
6,14,MAZOWIECKIE
7,16,OPOLSKIE
8,18,PODKARPACKIE
9,20,PODLASKIE


In [34]:
for i in np.arange(0,len(voivodeship_df.columns)):
    voivodeship_df[voivodeship_df.columns[i]] = \
    voivodeship_df[voivodeship_df.columns[i]].astype(str).str.strip().str.lower().str.replace('\t', '')

voivodeship_df

,WOJ,NAZWA
0,02,dolnośląskie
1,04,kujawsko-pomorskie
2,06,lubelskie
3,08,lubuskie
4,10,łódzkie
5,12,małopolskie
6,14,mazowieckie
7,16,opolskie
8,18,podkarpackie
9,20,podlaskie


In [26]:
np.arange(0, len(voivodeship_df.columns))

array([0, 1])

In [35]:
voivodeship_df = voivodeship_df.rename(columns={'WOJ': 'WOJ_KOD',
                                                'NAZWA': 'WOJ_NAZ'})

In [36]:
voivodeship_df

,WOJ_KOD,WOJ_NAZ
0,02,dolnośląskie
1,04,kujawsko-pomorskie
2,06,lubelskie
3,08,lubuskie
4,10,łódzkie
5,12,małopolskie
6,14,mazowieckie
7,16,opolskie
8,18,podkarpackie
9,20,podlaskie


In [37]:
voivodeship_df['WOJ_ID'] = np.arange(0, 16, 1)
voivodeship_df

,WOJ_KOD,WOJ_NAZ,WOJ_ID
0,02,dolnośląskie,0
1,04,kujawsko-pomorskie,1
2,06,lubelskie,2
3,08,lubuskie,3
4,10,łódzkie,4
5,12,małopolskie,5
6,14,mazowieckie,6
7,16,opolskie,7
8,18,podkarpackie,8
9,20,podlaskie,9
